In [1]:
import pandas as pd

print('Imports complete.')

Imports complete.


In [2]:
df = pd.read_csv('../../tor_dataset/Scenario-A/SelectedFeatures-10s-TOR-NonTOR.csv')
print(df.head())

   Source IP   Source Port  Destination IP   Destination Port   Protocol  \
0  10.0.2.15         53913   216.58.208.46                 80          6   
1  10.0.2.15         53913   216.58.208.46                 80          6   
2  10.0.2.15         53913   216.58.208.46                 80          6   
3  10.0.2.15         53913   216.58.208.46                 80          6   
4  10.0.2.15         53913   216.58.208.46                 80          6   

    Flow Duration   Flow Bytes/s   Flow Packets/s   Flow IAT Mean  \
0             435            0.0      4597.701149           435.0   
1             259            0.0      7722.007722           259.0   
2             891            0.0      2244.668911           891.0   
3            1074            0.0      1862.197393          1074.0   
4             315            0.0      6349.206349           315.0   

    Flow IAT Std  ...   Bwd IAT Min  Active Mean   Active Std   Active Max  \
0            0.0  ...             0            0  

In [3]:
print(df['label'].value_counts())

nonTOR    59790
TOR        8044
Name: label, dtype: int64


In [4]:
df = pd.concat([ (df.loc[ df['label'] == 'nonTOR']).sample(8044), df.loc[ df['label'] == 'TOR'] ])
print(df['label'].value_counts())

nonTOR    8044
TOR       8044
Name: label, dtype: int64


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

print('Imports complete.')

Imports complete.


In [6]:
random_state=1
dt = DecisionTreeClassifier(random_state=random_state)

In [7]:
for col in df.columns[:-1]:
    #print(X[col].dtype)
    if df[col].dtype == 'object':
        del df[col]

In [8]:
X = df.loc[:, df.columns != 'label']
y = df['label']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=random_state)

In [9]:
print('Length of X_train: {}'.format(len(X_train)))
print('Length of X_test: {}'.format(len(X_test)))

Length of X_train: 12870
Length of X_test: 3218


In [10]:
print(X_train)

        Source Port   Destination Port   Protocol   Flow Duration  \
33184          5639                 53         17          148504   
38781         18572                 53         17          132374   
55113         17500              17500         17            2270   
58788           443              54130          6         9997767   
58352           443              54130          6         9940213   
...             ...                ...        ...             ...   
60028           443              41994          6         9999763   
54057           443              37652          6         9970335   
56163         17500              17500         17            1308   
29157         54420                443          6         9494284   
55675         36922                443          6         9926164   

        Flow Bytes/s   Flow Packets/s   Flow IAT Mean   Flow IAT Std  \
33184    1178.419437        13.467651    1.485040e+05   0.000000e+00   
38781    1979.240636       

In [11]:
dt.fit(X_train, y_train)
results_dt = dt.score(X_test, y_test)
print(results_dt)

0.9801118707271598


In [13]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=random_state)
rf.fit(X_train, y_train)
results_rf = rf.score(X_test, y_test)
print(results_rf)

0.9869484151646986
